In [2]:
import numpy as np
import torch
from transformers import BertTokenizer, BertForMaskedLM
from tqdm import tqdm
import copy

from CWIs.complex_labeller import Complexity_labeller
from plainifier.plainify import *

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed = 1234
np.random.seed(seed)
torch.manual_seed(seed)

two_gram_mwes_list = './CWIs/2_gram_mwe_50.txt'
three_gram_mwes_list = './CWIs/3_gram_mwe_25.txt'
four_gram_mwes_list = './CWIs/4_gram_mwe_8.txt'
pretrained_model_path = './CWIs/cwi_seq.model'
temp_path = './CWIs/temp_file.txt'

path = './plainifier/'
premodel = 'bert-large-uncased-whole-word-masking'
bert_dict = 'tersebert_pytorch_1_0.bin'
embedding = 'crawl-300d-2M-subword.vec'
unigram = 'unigrams-df.tsv'
tokenizer = BertTokenizer.from_pretrained(premodel)
Complexity_labeller_model = Complexity_labeller(pretrained_model_path, temp_path)

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/daniel/Documents/MSc_ML_UCL/COMP0087/LSBert_new/CWIs/labeler.py:142: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  use_peepholes=self.config["lstm_use_peepholes"],
/home/daniel/Documents/MSc_ML_UCL/COMP0087/LSBert_new/CWIs/labeler.py:147: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  use_peepholes=self.config["lstm_use_peepholes"],
/home/daniel/.local/lib/python3.9/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:984: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._kernel = self.add_variable(
/home/daniel/.local/lib/python3.9/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.p

In [4]:
# loading bert model, word embeddings and unigrams. This process takes 7 minutes
model, similm, tokenfreq, embeddings, vocabulary2 = load_all(path, premodel, bert_dict, embedding, unigram, tokenizer)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading Embeddings


100%|██████████████████████████████| 2000000/2000000 [01:30<00:00, 22016.51it/s]


Loaded Embeddings
Loading Unigrams


100%|██████████████████████████████| 8394369/8394369 [06:04<00:00, 23058.09it/s]

Loaded Unigrams


In [38]:
class ComplexSentence:
    # Sentence class
    def __init__(self, sentence, label_model, tokeniser, verbose=True):
        self.sentence = sentence
        self.tokenised_sentence = tokeniser.tokenize(self.sentence)
        self.label_model = label_model
        self.verbose = verbose

        if self.verbose:
            print(f'Untokenised sentence: {self.sentence}')
            print(f'Tokenised sentence: {self.tokenised_sentence}')

        self.label_complex_words()

    def label_complex_words(self):
        # applying complexity labeller to the sentence

        Complexity_labeller.convert_format_string(self.label_model, self.sentence)
        self.bin_labels = Complexity_labeller.get_bin_labels(self.label_model)
        self.is_complex = True if np.sum(self.bin_labels) >= 1 else False
        self.probs = Complexity_labeller.get_prob_labels(self.label_model)
        
        self.complexity_ranking = np.argsort(self.probs)[::-1]
        self.most_complex_word = self.tokenised_sentence[self.complexity_ranking[0]]

        if self.verbose:
            print(f'Complex probs: {self.probs}')
            print(f'Binary complexity labels: {self.bin_labels}')
            
            if self.is_complex:
                print(f'\t Most complex word: {self.most_complex_word}')
        
        if not self.is_complex:
            print(f'\t Simplificaiton complete or no complex expression found')
    
    def find_MWEs_w_most_complex_word(self, n_gram, filepath):
        # finds the n-gram mwe of the most complex word in the sentence, if any
        # returns: mwe positions or complex word positions
        if not self.is_complex:
            raise ValueError('Sentence is not complex')
        
        complex_word_pos = self.complexity_ranking[0]

        if complex_word_pos - n_gram + 1 > 0:
            sliding_start = complex_word_pos - n_gram + 1
        else:
            sliding_start = 0
        
        if complex_word_pos + n_gram - 1 < len(self.complexity_ranking):
            sliding_end = complex_word_pos
        else:
            sliding_end = len(self.complexity_ranking) - n_gram

        with open(filepath, 'r') as f:
            mwes = set(f.read().split('\n')) # make set
            avg_mwe_complexity = 0
            for pos in range(sliding_start, sliding_end + 1):
                possible_mwe = ' '.join(self.tokenised_sentence[pos: pos + n_gram])
                
                if possible_mwe in mwes:
                    
                    if np.mean(self.probs[pos:pos+n_gram]) > avg_mwe_complexity:
                        avg_mwe_complexity = np.mean(self.probs[pos:pos+n_gram])
                        valid_mwes_idx = np.arange(pos, pos+n_gram, 1)
                        mwe_found = possible_mwe
                    else:
                        continue
                        
        if avg_mwe_complexity > 0:
            if self.verbose:
                print(f'\t Found complex expression: {mwe_found}')
            self.idx_to_plainify = valid_mwes_idx
        else:
            if self.verbose:
                print(f'\t Found complex word: {self.most_complex_word}')
            self.idx_to_plainify = [complex_word_pos]
        
        return self.idx_to_plainify
    
    def find_all_ngram_mwes(self):
        # give priority to longer MWEs
        pass
            
    
    def one_step_plainify(self):
        idx_start = self.idx_to_plainify[0]
        idx_end = self.idx_to_plainify[-1]+1
        print(f'\n \t Plainifying complex word or expression: {self.tokenised_sentence[idx_start:idx_end]}')
        processed_sentence = tokeniseUntokenise(self.sentence, tokenizer)
        forward_result = getTokenReplacement(processed_sentence, idx_start, len(self.idx_to_plainify), 
                                  tokenizer, model, similm, tokenfreq, embeddings, vocabulary2,
                                  verbose=False, backwards=False, maxDepth=3, maxBreadth=16, alpha=(1/9,6/9,2/9))
        backward_result = getTokenReplacement(processed_sentence, idx_start, len(self.idx_to_plainify),
                                  tokenizer, model, similm, tokenfreq, embeddings, vocabulary2, 
                                  verbose=False, backwards=True, maxDepth=3, maxBreadth=16, alpha=(1/9,6/9,2/9))
        words, scores = aggregateResults((forward_result, backward_result))
        print(f'Suggested top 5 subtitutions: {words[:5]}')
    
    def reset(self):
        pass
        
    def recursive_plainify(self):
        pass

In [44]:
input_sentence = "I took a sip of my coffee and kept working."

s = ComplexSentence(input_sentence, label_model=Complexity_labeller_model, tokeniser=tokenizer, verbose=True)

s.find_MWEs_w_most_complex_word(n_gram=4, filepath=four_gram_mwes_list)

s.one_step_plainify()

Untokenised sentence: I took a sip of my coffee and kept working.
Tokenised sentence: ['i', 'took', 'a', 'sip', 'of', 'my', 'coffee', 'and', 'kept', 'working', '.']
Complex probs: [8.8666166e-05, 0.00041433785, 0.00010313044, 0.77511847, 5.3620286e-05, 0.00036452108, 0.006510981, 7.626165e-05, 0.035020363, 0.0018819745, 4.129277e-05]
Binary complexity labels: [array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])]
	 Most complex word: sip
	 Found complex expression: took a sip of

 	 Plainifying complex word or expression: ['took', 'a', 'sip', 'of']
Suggested top 5 subtitutions: ['took', 'had', 'finished', 'drank', 'drank all']
